In [1]:
import datetime
import ffmpeg
import itertools
import math
import numpy as np
import time

from pathlib import Path

from plume_detect.breathecam import MotionDetector
from plume_detect.motion import Contour, Event, EventSpace, Motion
from plume_detect.view import View

In [ ]:
def batched(coll, n):
    it = iter(coll)

    while batch := tuple(itertools.islice(it, n)):
        yield batch
        
def make_video_div(src: str, metadata: list[str]) -> str:
   return f"""
<div class="video-container">
  <video src="{src}" loop muted playbackRate=0.5></video>
  <div class="label">{'<br>'.join(metadata)}</div>
</div>
"""

def write_mp4_video(src: str, video: np.ndarray):
    h, w = video.shape[1], video.shape[2]

    process = (
        ffmpeg
        .input("pipe:", format="rawvideo", pix_fmt="rgb24", s=f"{w}x{h}")
        .filter("pad", width="ceil(iw/2)*2", height="ceil(ih/2)*2", color="black")
        .output(src, pix_fmt="yuv420p", vcodec="libx264", r=12, loglevel="quiet")
        .overwrite_output()
        .run_async(pipe_stdin=True)
    )

    for frame in video:
        process.stdin.write(frame.tobytes())

    process.stdin.close()
    process.wait()

def write_video_html(path: str, videoHTML: str, title: str = "") -> str:
  html = f"""
<html>
  <head>
    <script src="js/jquery-3.7.1.min.js"></script>
    <style>
      video {{min-height:300px}}
      img {{height:300px}}
      body {{margin:0px}}
      #site-title {{
        margin-bottom: 5px;
        text-align: center;
        width: 100%;
      }}
      .label {{
        font-size: 17px;
      }}
      .video-container {{
        display: inline-flex;
        flex-flow: column nowrap;
        margin: 3px;
      }}
    </style>
  </head>
  <body>
  {f'<h1 id="site-title">{title}</h1>' if (title := title.strip()) else ''}
  {videoHTML}

    <script>
      var ggg;
      function fixupVideo(v) {{
        console.log(v);
        var d = v.wrap('<div/>');
        console.log(d);
        if (v.attr('playbackRate')) {{
          v[0].playbackRate = parseFloat($(v).attr('playbackRate'));
        }}
        if (v.attr('trimRight')) {{
          vvv = v;
          ddd = d;
        }}
      }}

      function init() {{
        console.log('init');
        let observer = new IntersectionObserver(
          (entries, observer) => {{
            for (entry of entries) {{
              if (entry.isIntersecting) {{
            console.log('play', entry.target);
                entry.target.play();
              }} else  {{
          console.log('pause', entry.target);
                entry.target.pause(); 
        }}
            }}
          }},
          {{threshold: 0}}
        );
        
        $('img,video').each(function(i,v){{
          fixupVideo($(v));
          console.log('setting up', v);
          observer.observe(v);
        }});
      }}

      $(init);
    </script>
  </body>
</html>
"""
  with open(path, "w") as htmlFile:
    htmlFile.write(html)

In [ ]:
def write_events_html(path: str, eventspace: EventSpace, videos: list[np.ndarray], *, title: str = "", label: str = "event", dir: str = ""):
    video_divs = []

    dir = dir.strip()

    if len(dir) > 0:
        dir = f"./scroll/videos/{dir[:-1] if dir.endswith('/') else dir}"
    else:
        dir = "./scroll/videos"

    Path(dir).mkdir(parents=True, exist_ok=True)
    
    for i, event in enumerate(eventspace.events):
        src = f"{dir}/{label}{i}.mp4"

        try:
            write_mp4_video(src, videos[i])
            size = videos[i].shape[0:3]

            metadata = [
                f"Video {i + 1}",
                f"{size[0]} frames starting at {event.frames[0]}",
                f"Size: {size[2]}x{size[1]}",
                f"Points: {len(event.contour)}",
                f"Volume: {event.region.width * event.region.height * len(event.frames)}", 
                f"Density: {event.density(5)}",
                f"Region Density: {event.region_density(5)}"
            ]

            video_divs.append(make_video_div(src, metadata))
        except BrokenPipeError as e:
            print(e)

    write_video_html(path, '\n'.join(video_divs), title)

In [ ]:
day = datetime.date.fromisoformat("2024-05-19")
nlevels = 4
neighborhood = 8
#view = View(4653, 2127, 4915, 2322)
view = View(2307, 1914, 6814, 2515)
view_ss = view.subsample(nlevels)
day_time = datetime.time.fromisoformat("09:49:00")

In [ ]:
def run_analysis(frames: int, detectShadows: bool, varThreshold: int, temporal_window: int = 3):    
    print(f"Size: {view.width}x{view.height} ({view.width * view.height} pixels)")
    print(f"Subsampled Size: {view_ss.width}x{view_ss.height} ({view_ss.width * view_ss.height})")
    print(f"Neighbors: {neighborhood}")
    print(f"Temporal Window: {temporal_window}")

    motion_detector = MotionDetector("Clairton Coke Works", day)
    motion_capture = motion_detector.analyze(day_time, view, frames, nlevels, detectShadows=detectShadows, varThreshold=varThreshold)
    eventspace = motion_capture.motion.event_space(neighborhood=8, span=temporal_window, points=100)
    full_res_video = motion_capture.download_video(nlevels=1)

    eventspace_sorted_by_points = eventspace.sorted_by_points(descending=True)

    write_events_html(
        f"events_{frames}_frames_threshold_{varThreshold}.html",
        eventspace_sorted_by_points,
        eventspace_sorted_by_points.upsampled_videos(
            nlevels, 
            full_res_video, 
            pad_frames=5, 
            pad_pixels=64),
        title = f"Events - {frames} frames - Threshold {varThreshold}",
        dir=f"events/threshold/{varThreshold}/frames/{frames}")
    
    print(f"Events Found: {len(eventspace)}\n")

In [ ]:
video_lengths = [1200]#, 200, 400]
thresholds = [16, 32, 64]

for n in video_lengths:
    #run_analysis(n, detectShadows=True, varThreshold=32)
    run_analysis(n, detectShadows=True, varThreshold=8)